In [1]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
from scipy.spatial import KDTree
from datasketch import MinHash, MinHashLSH
from glob import glob
from tqdm import tqdm
import numpy as np
import mp
import pandas as pd

In [4]:
# num_perm = 256
# threshold = 0.5

# def loop(files):
#     files, _ = files
#     lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

#     unique_embeddings = []

#     for f in tqdm(files):
#         emb = np.load(f)
#         m = MinHash(num_perm=num_perm)
#         for val in emb:
#             m.update(str(val).encode("utf8"))
#         if not lsh.query(m):
#             lsh.insert(str(len(unique_embeddings)), m)
#             unique_embeddings.append(f)
    
#     return unique_embeddings

# unique_files = mp.multiprocessing(files[:10000], loop, cores = 10)

In [5]:
files = glob('embedding/*.npy')
len(files)

9675

In [6]:
arrays = []
for f in tqdm(files):
    arrays.append(np.load(f))

100%|██████████████████████████████████████████████████████████████████████████████████| 9675/9675 [00:00<00:00, 22531.40it/s]


In [7]:
embeddings = np.array(arrays)
norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
normalized_embeddings = embeddings / norms

In [8]:
%%time

kd_tree = KDTree(normalized_embeddings, leafsize = 40)

CPU times: user 31.2 ms, sys: 6.26 ms, total: 37.5 ms
Wall time: 36.4 ms


In [9]:
dist, ind = kd_tree.query(arrays[0][None], k=len(arrays))

In [10]:
dist.shape

(1, 9675)

In [11]:
threshold = 0.2

In [12]:
visited = set()
unique_indices = []

for i in tqdm(range(len(arrays))):
    if i in visited:
        continue
    neighbors = kd_tree.query_ball_point(normalized_embeddings[i], threshold)
    if len(neighbors) > 1:
        print(i, neighbors)
    visited.update(neighbors)
    
    unique_indices.append(i)

100%|████████████████████████████████████████████████████████████████████████████████████| 9675/9675 [00:44<00:00, 215.92it/s]


In [13]:
df = pd.read_parquet('filtered-politicians.parquet')

In [21]:
import malaya_speech

`openai-whisper` is not available, native whisper processor is not available, will use huggingface processor instead.


In [22]:
model = malaya_speech.speaker_vector.nemo('huseinzol05/nemo-titanet_large').cuda()
_ = model.eval()